In [1]:
# Load library
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(os.path.join(PROJ_DIR,'src'))

import xai_faithfulness_experiments_lib_edits as ff
import numpy as np

In [18]:
DATASET = 'mnist'
SAMPLE_NUM = 50
MODEL_NAME = 'softmax'
GENERATION = '_genetic'

# Load data
data = ff.load_generated_data(os.path.join(PROJ_DIR, 'results', f'{DATASET}_{SAMPLE_NUM}_{MODEL_NAME}{GENERATION}_measures.npz'))
qmeans = data['qmeans']

In [19]:
print(np.mean(qmeans))
print(qmeans.shape)

0.84850603
(10000, 1)


In [20]:
GENERATION = ''

# Load data
data_reference = ff.load_generated_data(os.path.join(PROJ_DIR, 'results', f'{DATASET}_{SAMPLE_NUM}_{MODEL_NAME}{GENERATION}_measures.npz'))
qmeans_reference = data_reference['qmeans']

In [21]:
print(np.mean(qmeans_reference))
print(qmeans_reference.shape)

0.7393527
(10000, 1)


In [94]:
# Load dataset
import torch
import torchvision

batch_size = 256

MNIST_PATH = os.path.join(PROJ_DIR, 'data', 'mnist')

train_loader = torch.utils.data.DataLoader(
torchvision.datasets.MNIST(MNIST_PATH, train=True, download=True,
                            transform=torchvision.transforms.Compose([
                            torchvision.transforms.ToTensor(),
                            torchvision.transforms.Normalize(
                                (0.1307,), (0.3081,))
                            ])),
batch_size=batch_size, shuffle=True)

examples = enumerate(train_loader)
batch_idx, (x_train, y_train) = next(examples)

row = x_train[SAMPLE_NUM].clone().detach().to(device)
label = y_train[SAMPLE_NUM].clone().detach().to(device)

# Load model
MODEL_PATH = os.path.join(PROJ_DIR,'assets', 'models', f'{DATASET}-{MODEL_NAME}-mlp.pth')
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')
network = ff.load_pretrained_mnist_model(MODEL_PATH)


Using cuda:0


In [95]:
import genetic_generator as gg
num_rankings = qmeans.shape[0]
INPUT_SHAPE = row.shape
NUM_SAMPLES = 20
masking_values = torch.from_numpy(np.zeros(x_train.shape[1:])).float().to(device)
# Genetically optimized
def fitness(ranking:np.ndarray) -> float:
    measures = ff.get_measures_for_ranking(row, torch.tensor(ranking, dtype=torch.float32).to(device), label, network, num_samples=NUM_SAMPLES, with_inverse=False, with_random=False, masking_values=masking_values)
    return measures['mean']
all_rankings = gg.generate_rankings(num_rankings, INPUT_SHAPE, fitness, num_iterations = 1)

100%|███████████████████████████████████| 14000/14000 [00:06<00:00, 2089.79it/s]


1/1 - Avg. fitness 0.5607939958572388


In [96]:
from tqdm import tqdm
random_means = np.zeros(num_rankings) # To be randomly generated on the first loop
for i in tqdm(range(num_rankings)):
    random_means[i] = fitness(ff._get_random_ranking_row(row.shape).cpu().numpy()) # Random generation
np.mean(random_means)

100%|███████████████████████████████████| 10000/10000 [00:04<00:00, 2043.21it/s]


0.5617121496349573